In [1]:
import pandas as pd
import numpy as np
import sys
import re
import string

import time
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
display((dfT:=pd.read_csv('T.csv')).head())
display((dfF:=pd.read_csv('F.csv')).head())

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [3]:
dfT['Value']=1
dfF['Value']=0

In [4]:
display(dfT.head())
display(dfF.head())

,title,text,subject,date,Value
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


,title,text,subject,date,Value
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [5]:
display((data:=dfT.append(dfF)).head())
data.info()

,title,text,subject,date,Value
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


<class 'pandas.core.frame.DataFrame'>
Int64Index: 44898 entries, 0 to 23480
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44898 non-null  object
 1   text     44898 non-null  object
 2   subject  44898 non-null  object
 3   date     44898 non-null  object
 4   Value    44898 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 2.1+ MB


In [6]:
x=[*data.columns]
x=[(i[0].upper()+i[1:]).strip() for i in x]
data.columns=x

In [7]:
(data:=data.drop(columns=['Subject','Date'])).head()

,Title,Text,Value
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,1


In [8]:
Data=pd.DataFrame()
Data['Data']=data['Title']+data['Text']
Data['Value']=data['Value']
Data.head()

,Data,Value
0,"As U.S. budget fight looms, Republicans flip t...",1
1,U.S. military to accept transgender recruits o...,1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,1
3,FBI Russia probe helped by Australian diplomat...,1
4,Trump wants Postal Service to charge 'much mor...,1


In [9]:
Data=Data.sample(frac=1).reset_index(drop=True)
display(Data.head())

,Data,Value
0,Clinton sole presidential hopeful to attend As...,1
1,Aid group warns of death among Rohingya in Ban...,1
2,Trump's legal team spokesman resigns amid repo...,1
3,VAN JONES Tells ‘Mad’ Liberals To Go Hug Trump...,0
4,Clinton’s Devastating New Ad Puts Trump’s Whi...,0


In [44]:
X=Data['Data']
display(X[2])
Y=Data['Value']
display(Y[2])

"Trump's legal team spokesman resigns amid reported shake-upWASHINGTON (Reuters) - U.S. President Donald Trump’s outside legal team spokesman has resigned amid media reports the team is being reorganized as investigations continue into alleged Russian meddling in the U.S. election and possible collusion with Trump’s campaign.  Mark Corallo confirmed his departure on Friday in an emailed statement. His resignation comes amid reports regarding the role of Marc Kasowitz, Trump’s attorney handing the investigation into the Trump campaign’s possible ties to Russia.  NBC reported on Friday that Kasowitz was no longer leading the legal team but would remain on board to provide guidance. The New York Times reported late on Thursday that his “role ... will be significantly reduced.” A source familiar with the situation told Reuters on Friday that Kasowitz had not left the team. Separately, Trump is considering appointing Wall Street financier and long-time supporter Anthony Scaramucci as his Wh

1

In [40]:
def cleanText(text):
    cleaned = re.sub("[^'a-zA-Z0-9]"," ",text)
    lowered = cleaned.lower().strip()
    return lowered
(x_cleaned:=[cleanText(t) for t in X])[1]

'aid group warns of death among rohingya in bangladeshcox s bazar  bangladesh  reuters    rohingya refugees in bangladesh could die due to a lack of  food  shelter and water  given the huge numbers fleeing violence in myanmar  an aid agency warned on sunday  as authorities began moving people to camps to streamline the distribution of help  nearly 410 000 members of the rohingya muslim minority fled from myanmar s western rakhine state to bangladesh to escape a military offensive that the united nations has branded a  textbook example of ethnic cleansing    many people are arriving hungry  exhausted and with no food or water   mark pierce  bangladesh country director for the save the children aid agency said in a statement   i m particularly worried that the demand for food  shelter  water and basic hygiene support is not being met due to the sheer number of people in need  if families can t meet their basic needs  the suffering will get even worse and lives could be lost   bangladesh 

In [12]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(x_cleaned)
x_tokenized = tokenizer.texts_to_sequences(x_cleaned)

In [13]:
print(x_tokenized[0])

[68, 151, 3694, 2, 1897, 2311, 121, 232, 69, 35, 7, 379, 106, 68, 11, 1, 105, 151, 1571, 2, 1897, 4, 235, 159, 2359, 20, 4, 884, 132, 1488, 3, 2311, 221, 5, 1721, 18, 50, 2251, 2, 53, 162, 25, 1347, 291, 1, 2311, 1721, 121, 1725, 10, 612, 2726, 917, 893, 4, 166, 8, 2251, 2, 133, 2311, 221, 706, 2, 132, 175, 15, 9, 211, 8, 14, 43, 2667, 56, 151, 2, 1, 603, 6, 178, 34, 68, 17, 1, 105, 51, 2, 1439, 1, 2096, 3, 68, 7, 160, 1395, 803, 334, 5, 78, 3694, 909, 387, 116, 23, 2913, 5, 1, 92, 3, 78, 635, 2081, 73, 12, 92, 1217, 8, 13, 17, 2108, 2, 1897, 36, 5, 304, 409, 3, 1, 917, 893, 15, 78, 331, 2235, 7, 92, 116, 23, 1038, 1347, 291, 24, 1232, 4, 1893, 2, 68, 18, 50, 208, 4, 1386, 25, 3351, 245, 334, 10, 1, 160, 811, 10, 1, 890, 397, 151, 96, 936, 221, 1007, 53, 753, 2118, 6, 76, 184, 18, 295, 994, 5, 26, 159, 6, 3190, 2311, 221, 5, 1721, 28, 4, 3023, 166, 374, 60, 39, 321, 556, 141, 3, 1, 1223, 149, 2, 35, 7, 810, 34, 6, 367, 118, 30, 24, 44, 345, 1, 1215, 1837, 317, 6, 1, 88, 76, 33, 28, 4, 

In [14]:
length_array = [len(s) for s in x_tokenized]
SEQUENCE_LENGTH = int(np.quantile(length_array,0.75))
print(SEQUENCE_LENGTH)

485


In [15]:
x_padded = pad_sequences(x_tokenized,maxlen=SEQUENCE_LENGTH)

In [16]:
x_train,x_test,y_train,y_test = train_test_split(x_padded,Y,test_size=0.2,random_state=42)

In [17]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(35918, 485)
(8980, 485)
(35918,)
(8980,)


In [18]:
VOCAB_LENGTH = len(tokenizer.word_index) + 1
VECTOR_SIZE = 100

def getModel():
    """
    Returns a trainable Sigmoid Convolutional Neural Network
    """
    model = keras.Sequential()
    model.add(layers.Embedding(input_dim=VOCAB_LENGTH,
                               output_dim=VECTOR_SIZE,
                               input_length=SEQUENCE_LENGTH
                              ))
    
    model.add(layers.Conv1D(128,kernel_size=4))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation("relu"))
    model.add(layers.MaxPooling1D(2))
    
    model.add(layers.Conv1D(256,kernel_size=4))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation("relu"))
    model.add(layers.MaxPooling1D(2))
    
    model.add(layers.Conv1D(512,kernel_size=4))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation("relu"))
    model.add(layers.MaxPooling1D(2))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(1,activation="sigmoid"))
    
    model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])
    
    return model

In [19]:
model = getModel()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 485, 100)          14381300  
_________________________________________________________________
conv1d (Conv1D)              (None, 482, 128)          51328     
_________________________________________________________________
batch_normalization (BatchNo (None, 482, 128)          512       
_________________________________________________________________
activation (Activation)      (None, 482, 128)          0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 241, 128)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 238, 256)          131328    
_________________________________________________________________
batch_normalization_1 (Batch (None, 238, 256)          1

In [47]:
history = model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=3)

Epoch 1/3
1123/1123 [==============================] - 714s 635ms/step - loss: 0.0560 - accuracy: 0.9882 - val_loss: 0.0220 - val_accuracy: 0.9949
Epoch 2/3
1123/1123 [==============================] - 686s 611ms/step - loss: 0.0190 - accuracy: 0.9954 - val_loss: 0.0184 - val_accuracy: 0.9961
Epoch 3/3
1123/1123 [==============================] - 678s 604ms/step - loss: 0.0120 - accuracy: 0.9971 - val_loss: 0.0100 - val_accuracy: 0.9977


In [48]:
model.save_weights("trained_model.h5")

In [49]:
import pickle
with open("tokenizer.pickle",mode="wb") as F:
    pickle.dump(tokenizer,F)

In [50]:
import json
label_map = {0:"Fake",
             1:"Real"
            }

json.dump(label_map,open("label_map.json",mode="w"))


In [51]:
class DeployModel():
    
    def __init__(self,weights_path,tokenizer_path,seq_length,label_map_path):
        self.model = getModel()
        self.model.load_weights(weights_path)
        self.tokenizer = pickle.load(open(tokenizer_path,mode="rb"))
        self.seq_len = seq_length
        self.label_map = json.load(open(label_map_path))
    def _prepare_data(self,text):
        cleaned = cleanText(text)
        tokenized = self.tokenizer.texts_to_sequences([cleaned])
        padded = pad_sequences(tokenized,maxlen=self.seq_len)
        return padded
    def _predict(self,text):
        text = self._prepare_data(text)
        pred = int(self.model.predict_classes(text)[0])
        return str(pred)
    def result(self,text):
        pred = self._predict(text)
        return self.label_map[pred]

In [52]:
deploy_model = DeployModel(weights_path="./trained_model.h5",
                           tokenizer_path="./tokenizer.pickle",
                           seq_length=SEQUENCE_LENGTH,
                           label_map_path="./label_map.json"
                          )

In [53]:
deploy_model.result(cleanText(input()))

Trump's legal team spokesman resigns amid reported shake-upWASHINGTON (Reuters) - U.S. President Donald Trump’s outside legal team spokesman has resigned amid media reports the team is being reorganized as investigations continue into alleged Russian meddling in the U.S. election and possible collusion with Trump’s campaign.  Mark Corallo confirmed his departure on Friday in an emailed statement. His resignation comes amid reports regarding the role of Marc Kasowitz, Trump’s attorney handing the investigation into the Trump campaign’s possible ties to Russia.  NBC reported on Friday that Kasowitz was no longer leading the legal team but would remain on board to provide guidance. The New York Times reported late on Thursday that his “role ... will be significantly reduced.” A source familiar with the situation told Reuters on Friday that Kasowitz had not left the team. Separately, Trump is considering appointing Wall Street financier and long-time supporter Anthony Scaramucci as his Whi

'Real'

In [55]:
model=getModel()
model.load_weights("./trained_model.h5")
tokenizer = pickle.load(open("./tokenizer.pickle",mode="rb"))